# ME 7220 Assignment 2
Aidan Clark

Student No. 20191990

## Exercise 2.1

A car has an initial speed of $v_{max}$ (meters per second) and is located a distance $-p$ (meters) from a stopping location. If the car can produce a maximun deceleration of $d_{max}$ (meters per second squared), find the distance $s$ (meters) at which the brakes should be applied (as a function of $v_{max}$ and $d_{max}$) so that the car stops at the stopping line? Clearly state your assumptions.

In [ ]:
#@title #**Example 1: Openloop controller for pit stop** { run: "auto" }
#@markdown This simulator models the state propogation of vehicle approaching a pit stop. A maximum deceleration of $d\sim \mathcal{N}(5,0.5)m/s^2$ is applied at a stopping distance predetermined by a controller. Using the simulator given here determine a good stopping distance to appy the breaks through trial and error. Is the simulation deterministic or stochastic? What is the initial condition of this simulation?




import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# We maintain the following numpy vectors  dx = f(x,u,t,param,w) , y = h(x,param,v)

# 1. initialization - init_parameters() -  #TODO by STUDENT : update initialization
dt = 0.01 ;sig_n_u = 0.5 ;sig_n_p = 0.5 ;sig_n_v = 0.005; t_end = 20.0; 
max_decel = 5 ;max_vel = 10.0  ; min_vel = 0.05  # Due to friction there is a minumum velocity that can be achieved
max_decel_act = max_decel  + abs(sig_n_u*np.random.randn()+0.3)
p0 = -30.0  #m
v0 = max_vel 
y_p0 = p0 + sig_n_p*np.random.randn()
y_v0 = v0 + sig_n_v*np.random.randn()
X = []; T= []; U =[]; Y=[]; # Variables for data acquisition

# these vectors are needed to define a system model
x = np.array([p0,v0]) # state vector - variables
y = np.array([y_p0,y_v0]) # measurment vector - variables
param = np.array([dt])  # the parameter vector - constants

stopping_distance = -(max_vel**2)/(2*max_decel)  # change this to get the result
stopping_distance = -10 #@param {type:"slider", min:-30, max:1, step:1.0}

# 2. Simulation  
for t in np.arange(0,t_end,dt):
  # Controller -  open loop pre compute
  if x[0]>stopping_distance and x[1]>min_vel:
    u_ref = -max_decel
    u_ref_act = -max_decel_act
  else:
    u_ref = 0.0 
    u_ref_act =0.0 
  
  u = u_ref_act


  # READ CURRENT STATE X
  p = x[0]
  v = x[1]
  a = u

  # Apply System model
  p = p + v*dt + 0.5*(a)*dt**2
  v = v + (a)*dt
  if abs(v) <= min_vel: # deadzone for velocity
    v=0.0

  # Apply Measurement model
  y_p = p + sig_n_p*np.random.randn()
  y_v = v + sig_n_v*np.random.randn()

  x = np.array([p,v])
  y = np.array([y_p,y_v])
  u = u
  
  #Data Aquisition
  X.append(x) 
  T.append(t)
  Y.append(y)
  U.append(u)

# 4. visualization functions
print("Final position : ", x[0], " m")
fig1= plt.figure()
p = plt.plot(T, X)
plt.xlabel('time (s)')
plt.ylabel('Position /  Velocity')
plt.gca().legend(('Position','Velocity'))


#fig1 = plt.figure()
#p = plt.plot(T, Y)

fig1 = plt.figure()
p = plt.plot(T, U)
plt.xlabel('time (s)')
plt.ylabel('Input acceleration ')
print()

## Exercise 2.2

Write a code to simualte a a vehicle odometer which is modeled using the following equation.
$$y_v=v +\eta~~~~ \\ \eta \sim \mathcal{N}(0,\sigma_v)$$

with $p=-30m$, $v=10m/s$ and $\sigma_v = 0.1 m/s $. Assuming that the velocity is not changing for 100 time steps, plot the output of the velocity sensor $y_v$ for the corresponding 100 time steps. 

## Exercise 2.3

Using the simulation above comment on what happens to the system response (position and velocity tracking error) when each of the following parameters are changed.
 1. $K_p$
 2. $K_d$
 3. $K_i$

## Exercise 2.4

Consider the following LTI model and comment on
 
 \begin{equation} \begin{array}{c} 
\dot{\mathbf{x}} = A\mathbf{x} + B\mathbf{u} \\ 
\mathbf{y} = C\mathbf{x}
\end{array}\end{equation}

$$A=\begin{pmatrix}5 & 2 &2\\ 1 & 1 &0 \\ 1 & 2 &0\end{pmatrix}, ~B=\begin{pmatrix}1 & 0 \\ 0 & 0 \\0 &1 \end{pmatrix},~ C = I_3 $$

 a. Controlability of the model.

 b. Stability of the system with no inputs. (stability of the A matrix)

 c.Find stabilizing gains $K$ for the system 
 
 d.Plot the resulting response of all states for an arbitrary initial condition.

## Exercise 2.5

Express the model given in the previous Exercise 4 as a discrete model of the form:
$$\mathbf{x}_{k+1}=A_k\mathbf{x}_{k}+B_k\mathbf{u}_{k} \\
\mathbf{y}_{k}=C_k\mathbf{x}_{k}$$
Use a samplig time of $dt=0.001$. Comment on the stability of the discrete system with no inputs.

## Exercise 2.6

Design an LQR controller to stabilize the model given in exercise 4 with  Q=10∗I3  and  R=1 . Plot the resulting tragectory of the system for an arbitrary initial condition.

## Exercise 2.7

Consider the following linear system given in the code. Find a stabilizing LQR gain $K$ for the system by selecting a suitable Q and R Matrix. Tune the gains such that it achieve a settling time of less than 7 seconds. 

In [ ]:
# The state space model
A= np.array([[0, 1.0, 0.0, 0.0],[0, 0,0,0],[0, 0,0,1],[0, 0,0,0]])
B= np.array([[0,0], [1.0,0],[0,0], [0,1.0]]);
C= np.eye(4);



Kc =np.zeros([2,4])  # TODO STUDENT:  REPLACE THIS GAIN USING AN LQR CONTROLLER

print(Kc)
#the new autonoumous system becomes dx= (A-B*Kc)x
A_new=(A-B@Kc)

sys = ct.ss(A_new,[0,0,0,0],C,0)  # Define the system 
T, yout =ct.initial_response(sys,np.linspace(0.0, 7.0, num=50),[1,1,1,1])  # simulate the system model
#for 7 seconds using an initial condition of (1,2)
fig, axs = plt.subplots(2, 1)
axs[0].plot(T,yout[0]);axs[0].grid()
axs[1].plot(T,yout[1]);axs[1].grid()

## Exercise 2.8

Using the simulation above establish the region of attraction for the system using trial and error. A hand drawing of the phase plane plot generated using trials of the simulator is sufficient. Please use states $p_x$ and $\theta$ as the x and y axis of the phase plane plot.

Comment on how are these initial conditions ( outside region of attraction) handled in application?